# Решение 1 задания

### Считывание, очистка и обработка данных

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import os

In [2]:
print(os.listdir())

['.ipynb_checkpoints', 'main.ipynb', 'main2.ipynb', 'Тестовое задание на технического менеджера-стажера.docx', 'файл 1.txt', 'файл 2.csv']


In [3]:
df = pd.read_csv('файл 1.txt', delimiter = "\t")
df.columns = [i.lower() for i in df.columns]
df['login'] = df.login.str.split('login').str.join('').astype(int)
print(df.shape)
df.head()

(701827, 5)


,login,tid,microtasks,assigned_ts,closed_ts
0,0,190563850.0,4.0,2017-04-20 12:09:39,2017-04-20 13:13:01
1,0,190561754.0,1.0,2017-04-20 12:10:30,2017-04-20 12:28:29
2,0,190565906.0,4.0,2017-04-20 12:21:31,2017-04-20 13:30:10
3,0,190560246.0,1.0,2017-04-20 12:28:30,2017-04-20 13:13:56
4,0,190562168.0,2.0,2017-04-20 12:28:42,2017-04-20 13:14:50


In [4]:
print(df.dtypes)

login            int32
tid            float64
microtasks     float64
assigned_ts     object
closed_ts       object
dtype: object


In [5]:
# Столбцы assigned_ts и closed_ts переводим в тип datetimе
df['assigned_ts'] = pd.to_datetime(df['assigned_ts'])
df['closed_ts'] = pd.to_datetime(df['closed_ts'])
print(df.dtypes)

login                   int32
tid                   float64
microtasks            float64
assigned_ts    datetime64[ns]
closed_ts      datetime64[ns]
dtype: object


In [6]:
# Удаляем дублирующиеся строки
df = df.drop_duplicates()
print(df.shape)
df.head()

(680037, 5)


,login,tid,microtasks,assigned_ts,closed_ts
0,0,190563850.0,4.0,2017-04-20 12:09:39,2017-04-20 13:13:01
1,0,190561754.0,1.0,2017-04-20 12:10:30,2017-04-20 12:28:29
2,0,190565906.0,4.0,2017-04-20 12:21:31,2017-04-20 13:30:10
3,0,190560246.0,1.0,2017-04-20 12:28:30,2017-04-20 13:13:56
4,0,190562168.0,2.0,2017-04-20 12:28:42,2017-04-20 13:14:50


In [7]:
# Находим задачи с некорректным значением даты и удаляем их
mist = df[df.assigned_ts > df.closed_ts].index.tolist()
df[df.assigned_ts > df.closed_ts]

,login,tid,microtasks,assigned_ts,closed_ts
534453,585,197340894.0,1.0,2017-05-24 16:13:06,2017-05-24 16:00:57


In [8]:
df.drop(mist, inplace=True)
print(df.shape)

(680036, 5)


In [9]:
# Создаем столбец time_work - время работы асессора над заданием в секундах
df['time_work'] = df.closed_ts - df.assigned_ts
df['time_work'] = df.time_work.dt.total_seconds()
print(df.shape)
df.head()

(680036, 6)


,login,tid,microtasks,assigned_ts,closed_ts,time_work
0,0,190563850.0,4.0,2017-04-20 12:09:39,2017-04-20 13:13:01,3802.0
1,0,190561754.0,1.0,2017-04-20 12:10:30,2017-04-20 12:28:29,1079.0
2,0,190565906.0,4.0,2017-04-20 12:21:31,2017-04-20 13:30:10,4119.0
3,0,190560246.0,1.0,2017-04-20 12:28:30,2017-04-20 13:13:56,2726.0
4,0,190562168.0,2.0,2017-04-20 12:28:42,2017-04-20 13:14:50,2768.0


### Генерация метрик для итогового датасета

#### Итоговый датасет будет состоять из 5 метрик:
<ol>
<li>login - логин асессора (индекс датасета)</li>
<li>microtasks_num - общее количество микрозадач по каждому асессору</li>
<li>total_time_work - общее количество времени потраченное на выполнение всех задач</li>
<li>time_for_one - время потраченное на выполнение одного микрозадания</li>
<li>salary_for_microtask - оплата за выполнение одного микрозадания</li>
</ol>  

#### microtasks_num - общее количество микрозадач по каждому асессору

In [10]:
result = df.groupby('login').microtasks.sum().to_frame()
result.name = 'microtasks_num'
print(result.shape)
result.head()

(767, 1)


,microtasks
login,
0,2172.0
1,3513.0
2,7479.0
3,19056.0
4,366.0


#### total_time_work - общее количество времени потраченное на выполнение всех задач

In [11]:
total_time_work = df.groupby('login').time_work.sum()
total_time_work.name = 'total_time_work'
result = pd.merge(result, total_time_work, on='login', how='inner')
print(result.shape)
result.head()

(767, 2)


,microtasks,total_time_work
login,,
0,2172.0,1820211.0
1,3513.0,405881.0
2,7479.0,731167.0
3,19056.0,10186842.0
4,366.0,139385.0


#### time_for_one - время потраченное на выполнение одного микрозадания

In [12]:
# Для начала отбираем повторяющиеся задачи
tid_num = df.tid.value_counts()
repet_tid = tid_num[tid_num > 1].index.tolist() 
len(repet_tid)

10358

In [13]:
# Создаем датасет с повторяющимеся задачами
repet_tid = df[df.tid.isin(repet_tid)]
print(repet_tid.shape)
repet_tid.head()

(55350, 6)


,login,tid,microtasks,assigned_ts,closed_ts,time_work
11,0,189300086.0,4.0,2017-04-20 19:02:21,2017-04-20 19:28:59,1598.0
191,0,189530764.0,8.0,2017-04-23 17:30:51,2017-04-24 09:43:59,58388.0
200,0,166021510.0,2.0,2017-04-25 07:28:35,2017-04-25 07:46:42,1087.0
201,0,191696830.0,1.0,2017-04-25 07:37:47,2017-04-25 08:08:21,1834.0
221,0,191793036.0,2.0,2017-04-25 11:12:53,2017-04-25 11:25:13,740.0


In [14]:
# Отбираем самые быстрые решения по каждой задаче
real_time_work = repet_tid.groupby('tid').time_work.min() 
print(real_time_work.shape)
real_time_work.name = 'real_time_work'
real_time_work.head()

(10358,)


tid
113958765.0    139.0
113958927.0    297.0
113959029.0    150.0
113959131.0    202.0
113959365.0     50.0
Name: real_time_work, dtype: float64

In [15]:
# Создаем датасет где есть показатель real_time_work - самого быстрого решения по каждой задаче
a = pd.merge(df, real_time_work, on='tid', how='left')
a = a[a.real_time_work.notna()]
print(a.shape)
a.head()

(55350, 7)


,login,tid,microtasks,assigned_ts,closed_ts,time_work,real_time_work
11,0,189300086.0,4.0,2017-04-20 19:02:21,2017-04-20 19:28:59,1598.0,149.0
191,0,189530764.0,8.0,2017-04-23 17:30:51,2017-04-24 09:43:59,58388.0,579.0
200,0,166021510.0,2.0,2017-04-25 07:28:35,2017-04-25 07:46:42,1087.0,144.0
201,0,191696830.0,1.0,2017-04-25 07:37:47,2017-04-25 08:08:21,1834.0,20.0
221,0,191793036.0,2.0,2017-04-25 11:12:53,2017-04-25 11:25:13,740.0,156.0


In [16]:
# Вычисляем время потраченное на выполнение одного микрозадания¶
a['time_for_one'] = a.real_time_work / a.microtasks
print(a.shape)
a.head()

(55350, 8)


,login,tid,microtasks,assigned_ts,closed_ts,time_work,real_time_work,time_for_one
11,0,189300086.0,4.0,2017-04-20 19:02:21,2017-04-20 19:28:59,1598.0,149.0,37.250
191,0,189530764.0,8.0,2017-04-23 17:30:51,2017-04-24 09:43:59,58388.0,579.0,72.375
200,0,166021510.0,2.0,2017-04-25 07:28:35,2017-04-25 07:46:42,1087.0,144.0,72.000
201,0,191696830.0,1.0,2017-04-25 07:37:47,2017-04-25 08:08:21,1834.0,20.0,20.000
221,0,191793036.0,2.0,2017-04-25 11:12:53,2017-04-25 11:25:13,740.0,156.0,78.000


In [17]:
# Генерируем показатель time_for_one - сколько времени в среднем тратит на выполнение одного задания каждый асессор
time_for_one = a.groupby('login').time_for_one.mean()
print(time_for_one.shape)
time_for_one.head()

(766,)


login
0    48.607094
1    33.492291
2    33.510963
3    43.332131
4    28.967294
Name: time_for_one, dtype: float64

In [18]:
# Добавляем показатель time_for_one в итоговый датасет
result = pd.merge(result, time_for_one, on='login', how='left')
print(result.shape)
result.head()

(767, 3)


,microtasks,total_time_work,time_for_one
login,,,
0,2172.0,1820211.0,48.607094
1,3513.0,405881.0,33.492291
2,7479.0,731167.0,33.510963
3,19056.0,10186842.0,43.332131
4,366.0,139385.0,28.967294


In [19]:
# Проверка и заполнение пустых значений
result[result.time_for_one.isna()]

,microtasks,total_time_work,time_for_one
login,,,
282,1.0,25.0,NaN


In [20]:
change = result[result.time_for_one.isna()].index.tolist()
result.loc[change,'time_for_one'] = result.loc[change,'total_time_work']

#### salary_for_microtask - оплата за выполнение одного микрозадания

In [21]:
salary = float(input('Сколько рублей Вы готовы заплатить асессору за 30 секунд его рабочего времени '))
result['salary_for_microtask'] = (result.time_for_one / 30) * salary
print(result.shape)
result.head()

Сколько рублей Вы готовы заплатить асессору за 30 секунд его рабочего времени 1
(767, 4)


,microtasks,total_time_work,time_for_one,salary_for_microtask
login,,,,
0,2172.0,1820211.0,48.607094,1.620236
1,3513.0,405881.0,33.492291,1.116410
2,7479.0,731167.0,33.510963,1.117032
3,19056.0,10186842.0,43.332131,1.444404
4,366.0,139385.0,28.967294,0.965576
